In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from pyspark.sql import functions as F
from pyspark.sql.types import *

/tmp/ipykernel_49913/1286269870.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
sc.setCheckpointDir("/dados10t/datalake/raw/checkpoint_dir/")

In [3]:
df = spark.read.parquet("/dados10t/datalake/raw/omop/tabelas_input/tabela_pessoa_07072022_vigivac_nao_padronizada_v2.parquet/")

In [4]:
df.printSchema()

root
 |-- id_vigvac: long (nullable = true)
 |-- vacinacao: string (nullable = true)
 |-- vac_d_sexo: string (nullable = true)
 |-- vac_d_codibge: string (nullable = true)
 |-- vac_inconsistente: string (nullable = true)
 |-- vac_d_idade: double (nullable = true)
 |-- vac_d_racacor: string (nullable = true)
 |-- srag_cs_sexo: string (nullable = true)
 |-- srag_cs_etinia: string (nullable = true)
 |-- srag_cs_raca: short (nullable = true)
 |-- sg_sexo_2: string (nullable = true)
 |-- sg_racacor_2: string (nullable = true)
 |-- sg_raw_datanascimento: string (nullable = true)
 |-- srag_raw_dt_nasc: string (nullable = true)
 |-- vac_raw_paciente_datanascimento: string (nullable = true)



In [5]:
df.count() #188537327

188537327

In [6]:
188537327 - 183759941 # 4777386 adicionadas via srag e sg   #vac tem 3437 pessoas sem sexo somente

4777386

### ajustar sexo vacina

In [7]:
df.groupBy("vac_d_sexo").count().orderBy(F.desc("count")).show(300,False) 

+----------+--------+
|vac_d_sexo|count   |
+----------+--------+
|F         |95510810|
|M         |88245694|
|null      |4777386 |
|I         |3437    |
+----------+--------+



In [8]:
df = df.withColumn("vac_d_sexo_ok",F.when(F.col("vac_d_sexo") == "M","Masculino" ).otherwise(F.col("vac_d_sexo")) )
df = df.withColumn("vac_d_sexo_ok",F.when(F.trim(F.col("vac_d_sexo_ok")) == "F","Feminino" ).otherwise(F.col("vac_d_sexo_ok")) )
df = df.withColumn("vac_d_sexo_ok",F.when(F.col("vac_d_sexo_ok").isin("","I"),None ).otherwise(F.col("vac_d_sexo_ok")) )

df = df.withColumn("vac_d_racacor",F.trim("vac_d_racacor"))


In [9]:
df.groupBy("vac_d_sexo_ok").count().orderBy(F.desc("count")).show(300,False) 

+-------------+--------+
|vac_d_sexo_ok|count   |
+-------------+--------+
|Feminino     |95510810|
|Masculino    |88245694|
|null         |4780823 |
+-------------+--------+



### ajustar sexo srag

In [10]:
df.groupBy("srag_cs_sexo").count().orderBy(F.desc("count")).show(300,False) 

+------------+---------+
|srag_cs_sexo|count    |
+------------+---------+
|null        |186216768|
|M           |1254792  |
|F           |1065744  |
|I           |23       |
+------------+---------+



In [11]:
df = df.withColumn("srag_cs_sexo_ok",F.when(F.col("srag_cs_sexo") == "M","Masculino" ).otherwise(F.col("srag_cs_sexo")) )

df = df.withColumn("srag_cs_sexo_ok",F.when(F.trim(F.col("srag_cs_sexo_ok")) == "F","Feminino" ).otherwise(F.col("srag_cs_sexo_ok")) )

df = df.withColumn("srag_cs_sexo_ok",F.when(F.col("srag_cs_sexo_ok").isin("","I"),None ).otherwise(F.col("srag_cs_sexo_ok")) )


In [12]:
df.groupBy("srag_cs_sexo_ok").count().orderBy(F.desc("count")).show(300,False) 

+---------------+---------+
|srag_cs_sexo_ok|count    |
+---------------+---------+
|null           |186216791|
|Masculino      |1254792  |
|Feminino       |1065744  |
+---------------+---------+



### ajustar sexo SG

In [13]:
df.groupBy("sg_sexo_2").count().orderBy(F.desc("count")).show(300,False) 

+----------+---------+
|sg_sexo_2 |count    |
+----------+---------+
|null      |135428772|
|Feminino  |28535800 |
|Masculino |24522435 |
|Indefinido|50320    |
+----------+---------+



In [14]:
df = df.withColumn("sg_sexo_ok",F.when(F.col("sg_sexo_2").isin("","Indefinido"),None ).otherwise(F.col("sg_sexo_2")) )


In [15]:
df.groupBy("sg_sexo_ok").count().orderBy(F.desc("count")).show(300,False) 

+----------+---------+
|sg_sexo_ok|count    |
+----------+---------+
|null      |135479092|
|Feminino  |28535800 |
|Masculino |24522435 |
+----------+---------+



### ajustar raça SG/VACINA/srag

In [16]:
def sex_harmo(var,idioma="pt-br"): #
    if idioma == "pt-br":
        if var =="M" or var == "Masculino":
            return "Masculino"
        elif var == 'F' or var == "Feminino":
            return "Feminino"
    if idioma == "en":
        if var =="M" or var == "Masculino":
            return "Male"
        elif var == "F" or var == "Feminino":
            return "Female"
    
        
        
def raca_harmo(var,idioma="pt-br"): #
    if idioma == "pt-br":
        if var == "Branca":
            return "BRANCA"
        elif var == "Preta" or var == "Parda":
            return "PRETA"
        elif var == "Indigena":
            return "INDIGENA"
        elif var == "Amarela":
            return "AMARELA"
    elif idioma == "en":
        if var =="BRANCA":
            return "White"
        elif var == "PRETA" or var == "PARDA":
            return "Black"
        elif var == "INDIGENA":
            return "American Indian"
        elif var == "AMARELA":
            return "Asian"
    else:
        return "erro"
    
    
def fix_raca_srag(raca):
    raca = str(raca)
    if raca:
        if raca == "1":
            return "BRANCA"
        elif raca == "2":
            return "PRETA"
        elif raca == "3":
            return "AMARELA"
        elif raca == "4":
            return "PARDA"
        elif raca == "5":
            return "INDIGENA"
        
        

def sex_harmo(var,idioma="pt-br"): #
    if idioma == "pt-br":
        if var =="M" or var == "Masculino":
            return "Masculino"
        elif var == 'F' or var == "Feminino":
            return "Feminino"
    if idioma == "en":
        if var =="M" or var == "Masculino":
            return "Male"
        elif var == "F" or var == "Feminino":
            return "Female"
    

In [17]:
udf_sex_omop = F.udf(sex_harmo,StringType() )

udf_race_omop = F.udf(raca_harmo,StringType() )

udf_fix_raca_srag = F.udf(fix_raca_srag,StringType() )


udf_sex_omop = F.udf(sex_harmo,StringType() )

In [18]:
df.groupBy("sg_racacor_2").count().orderBy(F.desc("count")).show(300,False)

+-------------+---------+
|sg_racacor_2 |count    |
+-------------+---------+
|null         |145199541|
|Branca       |20966789 |
|Parda        |18222142 |
|Preta        |2261902  |
|Amarela      |1766380  |
|Indigena     |120571   |
|Não informado|2        |
+-------------+---------+



In [19]:
df.groupBy("vac_d_racacor").count().orderBy(F.desc("count")).show(300,False)

+-------------+--------+
|vac_d_racacor|count   |
+-------------+--------+
|BRANCA       |61517395|
|null         |49851954|
|PARDA        |42199385|
|AMARELA      |26604921|
|PRETA        |7806738 |
|INDIGENA     |556934  |
+-------------+--------+



In [20]:
df.groupBy("srag_cs_raca").count().orderBy(F.desc("count")).show(300,False)

+------------+---------+
|srag_cs_raca|count    |
+------------+---------+
|null        |186242208|
|1           |1025030  |
|4           |765110   |
|9           |377702   |
|2           |100300   |
|3           |22621    |
|5           |4356     |
+------------+---------+



In [21]:
#print(raca_harmo("Branca","pt-br"))

In [22]:
#df = df.withColumn("raca_omop_vac",udf_race_omop(F.trim(F.col("vac_d_racacor"))  , F.lit("pt-br") )   ) # o dado original 
df = df.withColumn("raca_omop_en_vac", udf_race_omop( F.trim(F.col("vac_d_racacor")),F.lit("en")))


df = df.withColumn("raca_omop_sg",udf_race_omop(F.col("sg_racacor_2")))
df = df.withColumn("raca_omop_en_sg", udf_race_omop("raca_omop_sg",F.lit("en")   ))



df = df.withColumn("raca_omop_srag",udf_fix_raca_srag(F.col("srag_cs_raca")))
df = df.withColumn("raca_omop_en_srag", udf_race_omop( "raca_omop_srag",F.lit("en") ))

In [23]:
#tratando SG dt
df = df.withColumn("sub_datanascimento",F.substring("sg_raw_datanascimento",1,10))
df = df.withColumn("dt_datanascimento_sg",F.to_date("sub_datanascimento","yyyy-MM-dd"))

#tratando srag data
df = df.withColumn("dt_datanascimento_srag",F.to_date("srag_raw_dt_nasc","dd/MM/yyyy"))

In [24]:

df = df.withColumn("sexo_omop_en_vac", udf_sex_omop( F.col("vac_d_sexo_ok"),F.lit("en") ))


df = df.withColumn("sexo_omop_en_sg", udf_sex_omop( "sg_sexo_ok",F.lit("en") ))


df = df.withColumn("sexo_omop_en_srag", udf_sex_omop( "srag_cs_sexo_ok",F.lit("en") )).checkpoint()

23/10/18 16:52:19 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [25]:
df.groupBy("raca_omop_srag").count().orderBy(F.desc("count")).show(300,False)

+--------------+---------+
|raca_omop_srag|count    |
+--------------+---------+
|null          |186619910|
|BRANCA        |1025030  |
|PARDA         |765110   |
|PRETA         |100300   |
|AMARELA       |22621    |
|INDIGENA      |4356     |
+--------------+---------+



In [26]:
df.groupBy("raca_omop_en_srag").count().orderBy(F.desc("count")).show(300,False)

+-----------------+---------+
|raca_omop_en_srag|count    |
+-----------------+---------+
|null             |186619910|
|White            |1025030  |
|Black            |865410   |
|Asian            |22621    |
|American Indian  |4356     |
+-----------------+---------+



In [27]:
df.groupBy("raca_omop_en_vac").count().orderBy(F.desc("count")).show(300,False)

+----------------+--------+
|raca_omop_en_vac|count   |
+----------------+--------+
|White           |61517395|
|Black           |50006123|
|null            |49851954|
|Asian           |26604921|
|American Indian |556934  |
+----------------+--------+



In [28]:
df.groupBy("raca_omop_sg").count().orderBy(F.desc("count")).show(300,False)

+------------+---------+
|raca_omop_sg|count    |
+------------+---------+
|null        |145199543|
|BRANCA      |20966789 |
|PRETA       |20484044 |
|AMARELA     |1766380  |
|INDIGENA    |120571   |
+------------+---------+



In [29]:
df.groupBy("raca_omop_en_sg").count().orderBy(F.desc("count")).show(300,False)

+---------------+---------+
|raca_omop_en_sg|count    |
+---------------+---------+
|null           |145199543|
|White          |20966789 |
|Black          |20484044 |
|Asian          |1766380  |
|American Indian|120571   |
+---------------+---------+



In [30]:
df.printSchema()

root
 |-- id_vigvac: long (nullable = true)
 |-- vacinacao: string (nullable = true)
 |-- vac_d_sexo: string (nullable = true)
 |-- vac_d_codibge: string (nullable = true)
 |-- vac_inconsistente: string (nullable = true)
 |-- vac_d_idade: double (nullable = true)
 |-- vac_d_racacor: string (nullable = true)
 |-- srag_cs_sexo: string (nullable = true)
 |-- srag_cs_etinia: string (nullable = true)
 |-- srag_cs_raca: short (nullable = true)
 |-- sg_sexo_2: string (nullable = true)
 |-- sg_racacor_2: string (nullable = true)
 |-- sg_raw_datanascimento: string (nullable = true)
 |-- srag_raw_dt_nasc: string (nullable = true)
 |-- vac_raw_paciente_datanascimento: string (nullable = true)
 |-- vac_d_sexo_ok: string (nullable = true)
 |-- srag_cs_sexo_ok: string (nullable = true)
 |-- sg_sexo_ok: string (nullable = true)
 |-- raca_omop_en_vac: string (nullable = true)
 |-- raca_omop_sg: string (nullable = true)
 |-- raca_omop_en_sg: string (nullable = true)
 |-- raca_omop_srag: string (nullabl

In [31]:
df = df.withColumn("sexo",F.coalesce("sexo_omop_en_srag","sexo_omop_en_vac","sexo_omop_en_sg") ) #ordem: srag -> vac -> sg
df = df.withColumn("raca",F.coalesce("raca_omop_en_srag","raca_omop_en_vac","raca_omop_en_sg") ) #ordem: srag -> vac -> sg


df = df.withColumn("gender_source_value",F.coalesce("srag_cs_sexo_ok","vac_d_sexo_ok","sg_sexo_ok") ) #ordem: srag -> vac -> sg
df = df.withColumn("race_source_value",F.coalesce("raca_omop_srag","vac_d_racacor","raca_omop_sg") ) #ordem: srag -> vac -> sg


df = df.withColumn("dt_datanascimento",F.coalesce("dt_datanascimento_srag","vac_raw_paciente_datanascimento","dt_datanascimento_sg") ) #ordem: srag -> vac -> sg

In [32]:
#df.filter(F.col("srag_raw_dt_nasc").isNotNull() ).select("dt_datanascimento","srag_raw_dt_nasc","sg_raw_datanascimento","vac_raw_paciente_datanascimento","dt_datanascimento_sg","dt_datanascimento_srag").show(100,False)

In [35]:
#df = df.withColumn("person_id",F.col("id_vigvac"))

In [33]:
person = df.select("id_vigvac","sexo","dt_datanascimento","raca","gender_source_value","race_source_value")

In [34]:
person.filter(F.col("dt_datanascimento").isNotNull() ).count()

187643209

In [35]:
person.filter(F.col("sexo").isNotNull() ).count()

188531871

In [36]:
person.filter(F.col("raca").isNotNull() ).count()

151281144

In [37]:
person.groupBy("sexo").count().orderBy(F.desc("count")).show(20,False) 

+------+--------+
|sexo  |count   |
+------+--------+
|Female|97691350|
|Male  |90840521|
|null  |5456    |
+------+--------+



In [38]:
person.groupBy("raca").count().orderBy(F.desc("count")).show(100,False) 

+---------------+--------+
|raca           |count   |
+---------------+--------+
|White          |67765452|
|Black          |56227881|
|null           |37256183|
|Asian          |26695911|
|American Indian|591900  |
+---------------+--------+



In [39]:
person.show()

+------------+------+-----------------+-----+-------------------+-----------------+
|   id_vigvac|  sexo|dt_datanascimento| raca|gender_source_value|race_source_value|
+------------+------+-----------------+-----+-------------------+-----------------+
|841813590016|  Male|       1970-10-21| null|          Masculino|             null|
|841813590017|  Male|       1959-10-05|White|          Masculino|           BRANCA|
|841813590018|  Male|       1975-12-27|White|          Masculino|           BRANCA|
|841813590019|  Male|       1951-08-22|Black|          Masculino|            PRETA|
|841813590020|Female|       1987-04-06|Black|           Feminino|            PARDA|
|841813590021|Female|       1985-12-21|White|           Feminino|           BRANCA|
|841813590022|Female|       1948-06-27|White|           Feminino|           BRANCA|
|841813590023|Female|       1996-08-26|Black|           Feminino|            PRETA|
|841813590024|  Male|       1983-02-13|White|          Masculino|           

In [40]:
#id_maper_sexo_raca = spark.read.csv("/home/juracy.santos/lista_termos_tb_pessoa_omop.csv",sep=";",header=True) #arquivo enviado para o mapeamento sexo/raça

In [41]:
sexo_raca_maper = spark.read.csv("/dados10t/datalake/standard/omop/resultado_mapeamento_sexo_raca.csv",sep=",",header=True)
gender = sexo_raca_maper.filter(F.col("target_vocabulary_id") == "Gender")
raca = sexo_raca_maper.filter(F.col("target_vocabulary_id") == "Race")

In [42]:
gender.show()

+-----------+-----------------+--------------------+-----------------------+-----------------+--------------------+----------------+--------------+--------------+
|source_code|source_concept_id|source_vocabulary_id|source_code_description|target_concept_id|target_vocabulary_id|valid_start_date|valid_end_date|invalid_reason|
+-----------+-----------------+--------------------+-----------------------+-----------------+--------------------+----------------+--------------+--------------+
|          0|                0|                   1|                 Female|             8532|              Gender|      1970-01-01|    2099-12-31|          null|
|          1|                0|                   1|                   Male|             8507|              Gender|      1970-01-01|    2099-12-31|          null|
+-----------+-----------------+--------------------+-----------------------+-----------------+--------------------+----------------+--------------+--------------+



In [43]:
#raca.show(5)

In [44]:
gender_v2 = gender.select('source_code_description','target_concept_id').withColumnRenamed("source_code_description","sexo").withColumnRenamed("target_concept_id","gender_concept_id")
raca_v2 = raca.select('source_code_description','target_concept_id').withColumnRenamed("source_code_description","raca").withColumnRenamed("target_concept_id","race_concept_id")

In [45]:
person_v2 = person.join(gender_v2,['sexo'],'left')
person_v2 = person_v2.join(raca_v2,['raca'],'left')

person_v2 = person_v2.withColumn("dt_datanascimento_v2",F.to_date("dt_datanascimento"))

person_v2 = person_v2.withColumn("year_of_birth",F.year("dt_datanascimento_v2"))
person_v2 = person_v2.withColumn("month_of_birth",F.month("dt_datanascimento_v2"))
person_v2 = person_v2.withColumn("day_of_birth",F.dayofmonth("dt_datanascimento_v2"))

person_v2 = person_v2.withColumn("birth_datetime", F.to_timestamp("dt_datanascimento_v2", "yyyy-MM-dd"))


person_v2 = person_v2.withColumn("ethnicity_concept_id",F.lit("38003563"))



In [46]:
#person_v2.show(10)

In [47]:
person_v3 = person_v2.drop(*['dt_datanascimento','dt_datanascimento_v2'])

In [48]:
person_v3.printSchema()

root
 |-- raca: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- id_vigvac: long (nullable = true)
 |-- gender_source_value: string (nullable = true)
 |-- race_source_value: string (nullable = true)
 |-- gender_concept_id: string (nullable = true)
 |-- race_concept_id: string (nullable = true)
 |-- year_of_birth: integer (nullable = true)
 |-- month_of_birth: integer (nullable = true)
 |-- day_of_birth: integer (nullable = true)
 |-- birth_datetime: timestamp (nullable = true)
 |-- ethnicity_concept_id: string (nullable = false)



In [49]:
#person_v3.show(10)

In [50]:
person_v3.filter(F.col("birth_datetime") < "1900-01-01").count()

4661

In [51]:
from pyspark.sql.window import Window

In [54]:
import time
start = time.time()

In [56]:
person_v4 = person_v3.rdd.zipWithIndex().toDF()


In [57]:
person_v5=person_v4.select(F.col("_1.*"),F.col("_2").alias('increasing_id'))

In [58]:
person_v5.printSchema()

root
 |-- raca: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- id_vigvac: long (nullable = true)
 |-- gender_source_value: string (nullable = true)
 |-- race_source_value: string (nullable = true)
 |-- gender_concept_id: string (nullable = true)
 |-- race_concept_id: string (nullable = true)
 |-- year_of_birth: long (nullable = true)
 |-- month_of_birth: long (nullable = true)
 |-- day_of_birth: long (nullable = true)
 |-- birth_datetime: timestamp (nullable = true)
 |-- ethnicity_concept_id: string (nullable = true)
 |-- increasing_id: long (nullable = true)



In [59]:
end = time.time()
print(end - start)

157.5714042186737


In [60]:
person_v5 = person_v5.withColumnRenamed("increasing_id","person_id")

In [61]:
person_v5.printSchema()

root
 |-- raca: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- id_vigvac: long (nullable = true)
 |-- gender_source_value: string (nullable = true)
 |-- race_source_value: string (nullable = true)
 |-- gender_concept_id: string (nullable = true)
 |-- race_concept_id: string (nullable = true)
 |-- year_of_birth: long (nullable = true)
 |-- month_of_birth: long (nullable = true)
 |-- day_of_birth: long (nullable = true)
 |-- birth_datetime: timestamp (nullable = true)
 |-- ethnicity_concept_id: string (nullable = true)
 |-- person_id: long (nullable = true)



In [55]:
#person_v3= person_v3.withColumn("new_column",F.lit("ABC"))
#w = Window().partitionBy('new_column').orderBy(F.lit('A'))
#person_v4 = person_v3.withColumn("person_id_v2",F.row_number().over(w)).drop("new_column")

In [62]:
spark.conf.set("spark.sql.parquet.int96RebaseModeInWrite", "CORRECTED") #arquivo somente sera lido em spark 3.0+

In [63]:
person_v5.write.parquet("/dados10t/datalake/standard/omop/tabelas_omop_cmd_5.4_v1/tb_person.parquet")  #falta preencher location nessa tabela

In [1]:
#person_v3.filter(F.col("birth_datetime") < "1900-01-01").show(100)